## Function Library

In [ ]:

# Library
import pandas as pd
import requests
import json
from configparser import ConfigParser

# Get Credentials
parser = ConfigParser()
_ = parser.read('notebook.cfg')

# Loading data into Dataframe
filename = 'database/partial.csv'
df = pd.read_csv(filename)

# Retrieve the Credentials
client_id, client_secret = parser.get('app_test', 'client_id'), parser.get('app_test', 'client_secret')

# Basic Song Retrieving Functions
def getToken(client_id, client_secret):
    # Get token
    url = "https://accounts.spotify.com/api/token"

    payload=f'client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials'
    headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cookie': '__Host-device_id=AQA1E_8u1HHZzuE7O9d_fV4YwanfNozg6q_mdfweHpowhpZGWG9saI45tX6W5no4APxdKSdgb0ZqexZNfHeYiXVZhLhlP5JYj0M; sp_tr=false'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    AccessToken = json.loads(response.text)['access_token']
    return AccessToken

# Access Token Retrieval
accessToken = getToken(client_id, client_secret)

def getAudioFeature(AccessToken, song_id):
    url = f"https://api.spotify.com/v1/audio-features/{song_id}"

    payload = {}
    headers = {
        'Authorization': f'Bearer {AccessToken}'
    }

    songAudioFeature = requests.get(url, headers=headers, data=payload).json()

    return songAudioFeature

# Song's Audio Feature
def querySong(song_id):
    songQueue = getAudioFeature(accessToken, song_id)
    querySong = [songQueue['danceability'],
            songQueue['energy'],
            songQueue['loudness'],
            songQueue['mode'],
            songQueue['speechiness'],
            songQueue['acousticness'],
            songQueue['instrumentalness'],
            songQueue['liveness'],
            songQueue['valence']]
    return querySong

Cosine Similarity


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the first 1000 songs in the dataset
df = pd.read_csv('database/partial.csv')
feature_vectors = []

# Only select several columns
columns = [8, 9, 11, 12, 13, 14, 15, 16, 17]
for row in range(len(df)):
    temp = []
    for column in columns:
        temp.append(df.iloc[row][column])
    feature_vectors.append(temp)

# Calculate cosine similarity matrix
cosine_sim_matrix = cosine_similarity(feature_vectors)

# Get song based on the ID
query_song = [querySong('0bC7GKnxh9W9JIvJ6HVWxc')]

# Calculate cosine similarity between the query song and all other songs
cosine_similarities = cosine_similarity(query_song, feature_vectors)

# Get the most similar song index
most_similar_song_index = np.argmax(cosine_similarities)

# # Print the most similar song index and its corresponding similarity score
# print("Most similar song index:", most_similar_song_index)
# print("Similarity score:", cosine_similarities[0, most_similar_song_index])

print(df.iloc[most_similar_song_index])